# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize  
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
#### changed below line
#tf.random.set_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
#### Added a dot infront of the path
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
####train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
####val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())

batch_size = 20

In [4]:
##### Initial phase test workload so that we can test the basic models and try overfitting
##train_doc = train_doc[:120]
##val_doc = val_doc[:40]

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
###### adding an extra parameter of the image_idx so that i don't have to compile this generator function again and again

def generator(source_path, folder_list, batch_size,img_cnt_idx=30):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #### initially chosing 30 images , will latre reduce it to 10 or 20 images
    img_idx = [x for x in range( (30 - img_cnt_idx ),30)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size   # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    ### Not trying cropping because the images in the videos are from extreme left to extreme right so don;t want to lose info
                    image_resized=imresize(image,(100,100,3))
                    
                    
                    ### Tried different normalization visually both looked same, but my local machine GPU crashed while running on first one so avoiding it for now
                    ##image_resized - np.percentile(image_resized,5)/ np.percentile(image_resized,95) - np.percentile(image_resized,5)
                    ##(image_resized[:,:,0])/255
                    
                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        rem_num_batches = len(folder_list)%batch_size 
        if (rem_num_batches != 0) :
            for batch in range(rem_num_batches): # we iterate over the number of batches
                batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
                for folder in range(batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                        
                        #crop the images and resize them. Note that the images are of 2 different shape 
                        #and the conv3D will throw error if the inputs in a batch have different shapes
                        image_resized=imresize(image,(100,100,3))
                        
                        batch_data[folder,idx,:,:,0] =  (image_resized[:,:,0])/255  #normalise and feed in the image
                        batch_data[folder,idx,:,:,1] =  (image_resized[:,:,1])/255  #normalise and feed in the image
                        batch_data[folder,idx,:,:,2] =  (image_resized[:,:,2])/255  #normalise and feed in the image
                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
#### Adding a dot infront of the path
train_path = './Project_data/train'
val_path = './Project_data/val'
####train_path = '/notebooks/storage/Final_data/Collated_training/train'
####val_path = '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10                             # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D,Conv2D,MaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

model = Sequential()

model.add(Conv3D(8,kernel_size=(3,3,3), padding='same',input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(1,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(1,3,3),padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 24, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 100, 100, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 24, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 12, 50, 50, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 12, 50, 50, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 50, 50, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 12, 50, 50, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
num_epochs = 20 
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)

In [29]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [30]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [31]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 9s - loss: 1.3175 - categorical_accuracy: 0.4281 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 155s 5s/step - loss: 1.3111 - categorical_accuracy: 0.4324 - val_loss: 1.0568 - val_categorical_accuracy: 0.5100

Epoch 00001: saving model to model_init_2020-09-1212_14_36.139518/model-00001-1.31107-0.43235-1.05684-0.51000.h5
Epoch 2/20
34/34 [==============================] - 61s 2s/step - loss: 1.1882 - categorical_accuracy: 0.4544 - val_loss: 1.0044 - val_categorical_accuracy: 0.5800

Epoch 00002: saving model to model_init_2020-09-1212_14_36.139518/model-00002-1.18819-0.45441-1.00436-0.58000.h5
Epoch 3/20
34/34 [==============================] - 61s 2s/step - loss: 1.0485 - categorical_accuracy: 0.5559 - val_loss: 1.0268 - val_categorical_accuracy: 0.5800

Epoch 00003: saving model to model_init_2020-09-1212_14_36.139518/model-00003-1.04846-0.55588-1.02682-0.58000.h5
Epoch 4/20
34/34 [==============================] - 59s 2s/step - loss: 0.8948 - categorical_accuracy: 0.6279 - val_loss: 0.8182 - val_categorical_accuracy: 0.7200

Epoch 00004

#### <font color =blue>So for a model with 6 layers and input of 16 images  and image files of (120,120) we have an accuracy of 84% in training and 38% </font>

#### <font color =blue>So for a model with 6 layers and input of 24 images and image files of (120,120)  we have an accuracy of 82% in training and 57% </font>

#### <font color =blue>So for a model with 6 layers and input of 24 images and image files of (100,100) we have an accuracy of 85% in training and 43% in validation</font>

#### <font color =blue>So for a model with 6 layers and input of 24 images and image files of (100,100) and num_epochs = 20 we have an accuracy of 97.65% in training and 78% in validation</font>

### <span style ='background:yellow'>Model 2: Reducing 1 layer</span>

In [32]:
#write your model here

model = Sequential()

model.add(Conv3D(8,kernel_size=(3,3,3), padding='same',input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(1,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 24, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_7 (Batch (None, 24, 100, 100, 8)   32        
_________________________________________________________________
activation_9 (Activation)    (None, 24, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 12, 50, 50, 8)     0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 12, 50, 50, 16)    3472      
_________________________________________________________________
batch_normalization_8 (Batch (None, 12, 50, 50, 16)    64        
_________________________________________________________________
activation_10 (Activation)   (None, 12, 50, 50, 16)    0         
__________

In [33]:
num_epochs = 20 
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

In [34]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/trainEpoch 1/20
 ; batch size = 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 11.2524 - categorical_accuracy: 0.2172

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 65s 2s/step - loss: 11.3334 - categorical_accuracy: 0.2147 - val_loss: 12.7908 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2020-09-1212_14_36.139518/model-00001-11.33340-0.21471-12.79075-0.18000.h5
Epoch 2/20
34/34 [==============================] - 60s 2s/step - loss: 11.5401 - categorical_accuracy: 0.2412 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2020-09-1212_14_36.139518/model-00002-11.54010-0.24118-12.41093-0.23000.h5
Epoch 3/20
34/34 [==============================] - 60s 2s/step - loss: 10.3415 - categorical_accuracy: 0.3088 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2020-09-1212_14_36.139518/model-00003-10.34147-0.30882-12.41093-0.23000.h5
Epoch 4/20
34/34 [==============================] - 59s 2s/step - loss: 10.6133 - categorical_accuracy: 0.2971 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

#### <font color =blue>By reducing the number of layers the accuracy also decreased : train to 28% and val to 36%</font>

#### <font color =blue>Even increasing the number of images didn't have much effect and the accuracy was down at train : 24% and val : 30%</font>

#### <font color =blue>with batch size 20 and epoch = 20 and 24 images at (100,100)  the accuracy was for train : 40% and val : 54% </font>

### <span style='background:yellow'>Model 3:   Trying another model</span>

In [39]:
#write your model here

model = Sequential()

model.add(Conv3D(8,kernel_size=(3,3,3), padding='same',input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(16, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(32, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(64, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))



#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 24, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_15 (Batc (None, 24, 100, 100, 8)   32        
_________________________________________________________________
activation_17 (Activation)   (None, 24, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 12, 50, 50, 8)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 12, 50, 50, 16)    3472      
_________________________________________________________________
batch_normalization_16 (Batc (None, 12, 50, 50, 16)    64        
_________________________________________________________________
activation_18 (Activation)   (None, 12, 50, 50, 16)    0         
__________

In [40]:
num_epochs = 20 
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

In [41]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 2.0454 - categorical_accuracy: 0.2797

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 66s 2s/step - loss: 2.0169 - categorical_accuracy: 0.2809 - val_loss: 3.2635 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-09-1212_14_36.139518/model-00001-2.01686-0.28088-3.26352-0.25000.h5
Epoch 2/20
34/34 [==============================] - 57s 2s/step - loss: 1.4183 - categorical_accuracy: 0.3544 - val_loss: 1.4160 - val_categorical_accuracy: 0.3600

Epoch 00002: saving model to model_init_2020-09-1212_14_36.139518/model-00002-1.41831-0.35441-1.41597-0.36000.h5
Epoch 3/20
34/34 [==============================] - 59s 2s/step - loss: 1.3240 - categorical_accuracy: 0.3956 - val_loss: 1.1466 - val_categorical_accuracy: 0.4300

Epoch 00003: saving model to model_init_2020-09-1212_14_36.139518/model-00003-1.32400-0.39559-1.14656-0.43000.h5
Epoch 4/20
34/34 [==============================] - 60s 2s/step - loss: 1.2760 - categorical_accuracy: 0.4118 - val_loss: 1.2450 - val_categorical_accuracy: 0.4300

Epoch 00004:

#### <font color =blue>with batch size 20 and epoch = 10 and 16 images the accuracy was for train : 56% and val : 16% </font>

#### <font color =blue>with batch size 20 and epoch = 10 and 24 images the accuracy was for train : 58% and val : 16% </font>

#### <font color =blue>with batch size 20 and epoch = 20 and 24 images(100,100) the accuracy was for train : 73% and val : 34%  even the models generated in between were not very impressive</font>

### <span style='background:yellow'>Model 4:  Trying another variation of model </span>

In [42]:
#write your model here

model = Sequential()

model.add(Conv3D(8,kernel_size=(3,3,3), padding='same',input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(32, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(64, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))



#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))


#softmax layer
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_24 (Conv3D)           (None, 24, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_22 (Batc (None, 24, 100, 100, 8)   32        
_________________________________________________________________
activation_24 (Activation)   (None, 24, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_20 (MaxPooling (None, 12, 50, 50, 8)     0         
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 12, 50, 50, 16)    3472      
_________________________________________________________________
batch_normalization_23 (Batc (None, 12, 50, 50, 16)    64        
_________________________________________________________________
activation_25 (Activation)   (None, 12, 50, 50, 16)    0         
__________

In [43]:
num_epochs = 20  
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

In [44]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  ./Project_data/train ; batch size = 20
 ./Project_data/val ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 12.7278 - categorical_accuracy: 0.1938

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 65s 2s/step - loss: 12.5717 - categorical_accuracy: 0.2044 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-09-1212_14_36.139518/model-00001-12.57168-0.20441-12.41093-0.23000.h5
Epoch 2/20
34/34 [==============================] - 55s 2s/step - loss: 12.8708 - categorical_accuracy: 0.2015 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2020-09-1212_14_36.139518/model-00002-12.87077-0.20147-12.41093-0.23000.h5
Epoch 3/20
34/34 [==============================] - 58s 2s/step - loss: 12.5626 - categorical_accuracy: 0.2206 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2020-09-1212_14_36.139518/model-00003-12.56263-0.22059-12.41093-0.23000.h5
Epoch 4/20
34/34 [==============================] - 58s 2s/step - loss: 12.8234 - categorical_accuracy: 0.2044 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

#### <font color =blue>with batch size 16 and epoch = 10 and 24 images the accuracy was for train : 46% and val : 54% </font>

#### <font color =blue>with batch size 20 and epoch = 20 and 24 images(100,100) the accuracy was for train : 20% and val : 23%</font> 

### <span style='background:yellow'>Model 5:  Trying another variation of model </span>

In [14]:
#write your model here

model = Sequential()

model.add(Conv3D(8,kernel_size=(3,3,3), padding='same',input_shape=(30,100,100,3)))   #### 30 in case of full , image size 120,120
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(64, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))



#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 30, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_9 (Batch (None, 30, 100, 100, 8)   32        
_________________________________________________________________
activation_10 (Activation)   (None, 30, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 15, 50, 50, 8)     0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 15, 50, 50, 16)    3472      
_________________________________________________________________
batch_normalization_10 (Batc (None, 15, 50, 50, 16)    64        
_________________________________________________________________
activation_11 (Activation)   (None, 15, 50, 50, 16)    0         
__________

In [15]:
num_epochs = 50 
img_cnt_idx=30
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [16]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val Source path =  ./Project_data/train ; batch size = 20
; batch size = 20
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 4s - loss: 2.1835 - categorical_accuracy: 0.2703

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 80s 2s/step - loss: 2.1333 - categorical_accuracy: 0.2794 - val_loss: 1.7025 - val_categorical_accuracy: 0.2600

Epoch 00001: saving model to model_init_2020-09-1306_06_01.454275/model-00001-2.13326-0.27941-1.70250-0.26000.h5
Epoch 2/50
34/34 [==============================] - 74s 2s/step - loss: 1.4066 - categorical_accuracy: 0.3647 - val_loss: 1.2353 - val_categorical_accuracy: 0.4700

Epoch 00002: saving model to model_init_2020-09-1306_06_01.454275/model-00002-1.40664-0.36471-1.23529-0.47000.h5
Epoch 3/50
34/34 [==============================] - 73s 2s/step - loss: 1.3666 - categorical_accuracy: 0.3809 - val_loss: 1.1969 - val_categorical_accuracy: 0.5100

Epoch 00003: saving model to model_init_2020-09-1306_06_01.454275/model-00003-1.36658-0.38088-1.19695-0.51000.h5
Epoch 4/50
34/34 [==============================] - 74s 2s/step - loss: 1.3183 - categorical_accuracy: 0.3662 - val_loss: 1.1720 - val_categorical_accuracy: 0.3900

Epoch 00004:

#### <font color =blue>with batch size 20 and epoch = 10 and 24 images the accuracy was for train : 78% and val : 51% </font>

#### <font color = blue>with batch size 20 and epoch = 10 and 24 images at (100,100)  the accuracy was for train : 85% and val : 82% </font>

#### <font color = blue>with batch size 20 and epoch = 30 and 24 images at (100,100)  the accuracy was for train : 95% and val : 82% </font>

#### <font color = blue>with batch size 20 and epoch = 30 and 16 images at (100,100)  the accuracy was for train : 69% and val : 73% </font>

#### <font color = blue>with batch size 20 and epoch = 30 and 30 images at (100,100)  the accuracy was for train : 98% and val : 82% </font>

#### <font color = blue>with batch size 20 and epoch = 50 and 30 images at (100,100)  the accuracy was for train : 97.5% and val : 83%. So even with a lot of increase in epochs there was not much gain in the validation accuracy </font>

#### <font color =red>this looks promising so will train it for more epochs</font>

### <span style='background:yellow'>Model 5a):  Trying another variation of model, kernels changed </span>

In [19]:
#write your model here

model = Sequential()

model.add(Conv3D(8,kernel_size=(2,2,2), padding='same',input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, kernel_size=(2,2,2),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(2,2,2),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(2,2,2),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(64, kernel_size=(1,1,1),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))



#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_20 (Conv3D)           (None, 24, 100, 100, 8)   200       
_________________________________________________________________
batch_normalization_19 (Batc (None, 24, 100, 100, 8)   32        
_________________________________________________________________
activation_20 (Activation)   (None, 24, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 12, 50, 50, 8)     0         
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 12, 50, 50, 16)    1040      
_________________________________________________________________
batch_normalization_20 (Batc (None, 12, 50, 50, 16)    64        
_________________________________________________________________
activation_21 (Activation)   (None, 12, 50, 50, 16)    0         
__________

In [22]:
num_epochs = 40 
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [23]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 0.3276 - categorical_accuracy: 0.8844

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 63s 2s/step - loss: 0.3309 - categorical_accuracy: 0.8809 - val_loss: 0.5565 - val_categorical_accuracy: 0.8000

Epoch 00001: saving model to model_init_2020-09-1306_06_01.454275/model-00001-0.33089-0.88088-0.55651-0.80000.h5
Epoch 2/40
34/34 [==============================] - 60s 2s/step - loss: 0.3476 - categorical_accuracy: 0.8632 - val_loss: 0.5999 - val_categorical_accuracy: 0.8000

Epoch 00002: saving model to model_init_2020-09-1306_06_01.454275/model-00002-0.34756-0.86324-0.59992-0.80000.h5
Epoch 3/40
34/34 [==============================] - 60s 2s/step - loss: 0.3224 - categorical_accuracy: 0.8882 - val_loss: 0.6359 - val_categorical_accuracy: 0.8200

Epoch 00003: saving model to model_init_2020-09-1306_06_01.454275/model-00003-0.32236-0.88824-0.63587-0.82000.h5
Epoch 4/40
34/34 [==============================] - 61s 2s/step - loss: 0.3070 - categorical_accuracy: 0.8809 - val_loss: 0.5459 - val_categorical_accuracy: 0.8100

Epoch 00004:

#### <font color =blue>with batch size 20 and epoch = 20 and 24 images the accuracy was for train : 88% and val : 81% </font>

#### <font color =blue>with batch size 20 and epoch = 40 and 24 images the accuracy was for train : 93.6% and val : 85% </font>

### <span style='background:yellow'>Model 5b):  Trying another variation of model further changes in the kernel </span>

In [24]:
#write your model here

model = Sequential()

model.add(Conv3D(16,kernel_size=(1,1,1), padding='same',input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(2,2,2),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(128, kernel_size=(1,1,1),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(128, kernel_size=(1,1,1),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))



#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_25 (Conv3D)           (None, 24, 100, 100, 16)  64        
_________________________________________________________________
batch_normalization_24 (Batc (None, 24, 100, 100, 16)  64        
_________________________________________________________________
activation_25 (Activation)   (None, 24, 100, 100, 16)  0         
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 12, 50, 50, 16)    0         
_________________________________________________________________
conv3d_26 (Conv3D)           (None, 12, 50, 50, 32)    4128      
_________________________________________________________________
batch_normalization_25 (Batc (None, 12, 50, 50, 32)    128       
_________________________________________________________________
activation_26 (Activation)   (None, 12, 50, 50, 32)    0         
__________

In [25]:
num_epochs = 40 
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [26]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 3.5153 - categorical_accuracy: 0.2609

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 66s 2s/step - loss: 3.4107 - categorical_accuracy: 0.2632 - val_loss: 2.0034 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-09-1306_06_01.454275/model-00001-3.41070-0.26324-2.00337-0.23000.h5
Epoch 2/40
34/34 [==============================] - 58s 2s/step - loss: 1.5540 - categorical_accuracy: 0.3250 - val_loss: 1.4127 - val_categorical_accuracy: 0.3900

Epoch 00002: saving model to model_init_2020-09-1306_06_01.454275/model-00002-1.55396-0.32500-1.41267-0.39000.h5
Epoch 3/40
34/34 [==============================] - 61s 2s/step - loss: 1.3768 - categorical_accuracy: 0.3691 - val_loss: 1.4704 - val_categorical_accuracy: 0.3700

Epoch 00003: saving model to model_init_2020-09-1306_06_01.454275/model-00003-1.37684-0.36912-1.47039-0.37000.h5
Epoch 4/40
34/34 [==============================] - 61s 2s/step - loss: 1.2511 - categorical_accuracy: 0.4750 - val_loss: 1.1173 - val_categorical_accuracy: 0.5900

Epoch 00004:

#### <font color =blue>with batch size 20 and epoch = 40 and 24 images the accuracy was for train : 92.06% and val : 85% </font>

### <span style='background:yellow'>Model 5c):  Trying another variation of model , changed the dropout value </span>

In [12]:
#write your model here

model = Sequential()

model.add(Conv3D(16,kernel_size=(1,1,1), padding='same',input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(2,2,2),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(3,3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(128, kernel_size=(1,1,1),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(128, kernel_size=(1,1,1),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))



#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

#softmax layer
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 24, 100, 100, 16)  64        
_________________________________________________________________
batch_normalization_9 (Batch (None, 24, 100, 100, 16)  64        
_________________________________________________________________
activation_10 (Activation)   (None, 24, 100, 100, 16)  0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 12, 50, 50, 16)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 12, 50, 50, 32)    4128      
_________________________________________________________________
batch_normalization_10 (Batc (None, 12, 50, 50, 32)    128       
_________________________________________________________________
activation_11 (Activation)   (None, 12, 50, 50, 32)    0         
__________

In [13]:
num_epochs = 40 
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [14]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 9s - loss: 2.1246 - categorical_accuracy: 0.3016 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 161s 5s/step - loss: 2.0799 - categorical_accuracy: 0.3118 - val_loss: 2.6232 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-09-1311_47_18.067053/model-00001-2.07990-0.31176-2.62316-0.25000.h5
Epoch 2/40
34/34 [==============================] - 60s 2s/step - loss: 1.2780 - categorical_accuracy: 0.4691 - val_loss: 1.1705 - val_categorical_accuracy: 0.5100

Epoch 00002: saving model to model_init_2020-09-1311_47_18.067053/model-00002-1.27799-0.46912-1.17047-0.51000.h5
Epoch 3/40
34/34 [==============================] - 58s 2s/step - loss: 1.0359 - categorical_accuracy: 0.5824 - val_loss: 1.1558 - val_categorical_accuracy: 0.4700

Epoch 00003: saving model to model_init_2020-09-1311_47_18.067053/model-00003-1.03594-0.58235-1.15582-0.47000.h5
Epoch 4/40
34/34 [==============================] - 59s 2s/step - loss: 0.8519 - categorical_accuracy: 0.6676 - val_loss: 0.8102 - val_categorical_accuracy: 0.6600

Epoch 00004

#### <font color =blue>with batch size 20 and epoch = 40 and 24 images the best accuracy was for train : 98.09% and val : 82% </font>

### <span style='background:yellow'>Model 5d):  Trying another variation of model , changed the dropout value ,different kernel </span>

In [15]:
#write your model here

model = Sequential()

model.add(Conv3D(8,kernel_size=(2,2,2), padding='same',input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, kernel_size=(2,2,2),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(2,2,2),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(2,2,2),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(64, kernel_size=(1,1,1),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))



#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))

#softmax layer
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_15 (Conv3D)           (None, 24, 100, 100, 8)   200       
_________________________________________________________________
batch_normalization_14 (Batc (None, 24, 100, 100, 8)   32        
_________________________________________________________________
activation_15 (Activation)   (None, 24, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 12, 50, 50, 8)     0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 12, 50, 50, 16)    1040      
_________________________________________________________________
batch_normalization_15 (Batc (None, 12, 50, 50, 16)    64        
_________________________________________________________________
activation_16 (Activation)   (None, 12, 50, 50, 16)    0         
__________

In [17]:
num_epochs = 40 
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [18]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 2.1741 - categorical_accuracy: 0.2766

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 64s 2s/step - loss: 2.1469 - categorical_accuracy: 0.2750 - val_loss: 1.7896 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2020-09-1311_47_18.067053/model-00001-2.14694-0.27500-1.78960-0.29000.h5
Epoch 2/40
34/34 [==============================] - 58s 2s/step - loss: 1.3201 - categorical_accuracy: 0.4088 - val_loss: 1.2880 - val_categorical_accuracy: 0.4300

Epoch 00002: saving model to model_init_2020-09-1311_47_18.067053/model-00002-1.32007-0.40882-1.28802-0.43000.h5
Epoch 3/40
34/34 [==============================] - 58s 2s/step - loss: 1.1809 - categorical_accuracy: 0.4779 - val_loss: 1.0853 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_init_2020-09-1311_47_18.067053/model-00003-1.18092-0.47794-1.08525-0.55000.h5
Epoch 4/40
34/34 [==============================] - 59s 2s/step - loss: 1.0896 - categorical_accuracy: 0.5324 - val_loss: 1.2190 - val_categorical_accuracy: 0.4400

Epoch 00004:

#### <font color =blue>with batch size 20 and epoch = 40 and 24 images the best accuracy was for train : 99.26% and val : 83% </font>

## CNN + RNN

### <span style ='background:yellow'>Model 6 : Using a LSTM model</span>

In [47]:
model = Sequential()
model.add(TimeDistributed(Conv2D(8,kernel_size=(3,3), padding='same', activation='relu'),input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
        
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))


model.add(TimeDistributed(Flatten()))

model.add(LSTM(32))
model.add(Dropout(0.5))
        
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))

In [48]:
optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_5 (TimeDist (None, 24, 100, 100, 8)   224       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 24, 100, 100, 8)   32        
_________________________________________________________________
time_distributed_7 (TimeDist (None, 24, 50, 50, 8)     0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 24, 20000)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                2564224   
_________________________________________________________________
dropout_16 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                2112      
__________

In [49]:
batch_size = 20
num_epochs = 20  
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [50]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [51]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20
 ./Project_data/val ; batch size = 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 1.7055 - categorical_accuracy: 0.2266

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 64s 2s/step - loss: 1.7066 - categorical_accuracy: 0.2250 - val_loss: 1.6075 - val_categorical_accuracy: 0.2700

Epoch 00001: saving model to model_init_2020-09-1212_14_36.139518/model-00001-1.70660-0.22500-1.60750-0.27000.h5
Epoch 2/20
34/34 [==============================] - 58s 2s/step - loss: 1.5963 - categorical_accuracy: 0.2647 - val_loss: 1.6112 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2020-09-1212_14_36.139518/model-00002-1.59631-0.26471-1.61120-0.25000.h5
Epoch 3/20
34/34 [==============================] - 59s 2s/step - loss: 1.5307 - categorical_accuracy: 0.3191 - val_loss: 1.6309 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2020-09-1212_14_36.139518/model-00003-1.53067-0.31912-1.63088-0.21000.h5
Epoch 4/20
34/34 [==============================] - 59s 2s/step - loss: 1.5146 - categorical_accuracy: 0.3324 - val_loss: 1.4976 - val_categorical_accuracy: 0.3700

Epoch 00004:

#### <font color =blue>With batch size 20 and epoch = 10 and 16 images the accuracy was for train : 35% and val : 31% </font>

#### <font color =blue>With batch size 20 and epoch = 10 and 24 images the accuracy was for train : 42% and val : 48% </font>

#### <font color =blue>With batch size 16 and epoch = 20 and 24 images the accuracy was for train : 46% and val : 54% </font>

#### <font color =blue>With batch size 16 and epoch = 20 and 24 images the accuracy was for train : 46% and val : 45%  and time taken : 56 mins</font>

#### <font color =blue>With batch size 20 and epoch = 20 and 24 images(100,100) the accuracy was for train : 62% and val : 46%  </font>

### <span style='background:yellow'>Model 7: Adding another layer in the LSTM model</span>

In [54]:
model = Sequential()
model.add(TimeDistributed(Conv2D(8,kernel_size=(3,3), padding='same', activation='relu'),input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
        
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(32))
model.add(Dropout(0.5))
        
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_9 (TimeDist (None, 24, 100, 100, 8)   224       
_________________________________________________________________
time_distributed_10 (TimeDis (None, 24, 100, 100, 8)   32        
_________________________________________________________________
time_distributed_11 (TimeDis (None, 24, 50, 50, 8)     0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 24, 50, 50, 16)    1168      
_________________________________________________________________
time_distributed_13 (TimeDis (None, 24, 50, 50, 16)    64        
_________________________________________________________________
time_distributed_14 (TimeDis (None, 24, 25, 25, 16)    0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 24, 10000)         0         
__________

In [55]:
num_epochs = 20 
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]



if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [56]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 1.6441 - categorical_accuracy: 0.2406

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 63s 2s/step - loss: 1.6332 - categorical_accuracy: 0.2485 - val_loss: 1.5366 - val_categorical_accuracy: 0.3400

Epoch 00001: saving model to model_init_2020-09-1212_14_36.139518/model-00001-1.63320-0.24853-1.53662-0.34000.h5
Epoch 2/20
34/34 [==============================] - 56s 2s/step - loss: 1.5911 - categorical_accuracy: 0.2544 - val_loss: 1.5405 - val_categorical_accuracy: 0.2900

Epoch 00002: saving model to model_init_2020-09-1212_14_36.139518/model-00002-1.59106-0.25441-1.54050-0.29000.h5
Epoch 3/20
34/34 [==============================] - 57s 2s/step - loss: 1.6161 - categorical_accuracy: 0.2456 - val_loss: 1.5920 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2020-09-1212_14_36.139518/model-00003-1.61614-0.24559-1.59203-0.23000.h5
Epoch 4/20
34/34 [==============================] - 58s 2s/step - loss: 1.5722 - categorical_accuracy: 0.2971 - val_loss: 1.5596 - val_categorical_accuracy: 0.2800

Epoch 00004:

#### <font color =blue>With batch size 20 and epoch = 10 and 16 images the accuracy was for train : 48% and val : 29% </font>

#### <font color =blue>With batch size 20 and epoch = 20 and 24 images(100,100) the accuracy was for train : 51% and val : 55% </font>

### <span style='background:yellow'>Model 8: Using GRU</span>

In [57]:
model = Sequential()
model.add(TimeDistributed(Conv2D(8,kernel_size=(3,3), padding='same', activation='relu'),input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120
        
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))


model.add(TimeDistributed(Flatten()))

model.add(GRU(32))
model.add(Dropout(0.5))
        
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))

In [58]:
optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_16 (TimeDis (None, 24, 100, 100, 8)   224       
_________________________________________________________________
time_distributed_17 (TimeDis (None, 24, 100, 100, 8)   32        
_________________________________________________________________
time_distributed_18 (TimeDis (None, 24, 50, 50, 8)     0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 24, 20000)         0         
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                1923168   
_________________________________________________________________
dropout_20 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 64)                2112      
__________

In [59]:
#batch_size = 20
num_epochs = 20  
img_cnt_idx=24
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [60]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [61]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 1.7995 - categorical_accuracy: 0.2531

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 64s 2s/step - loss: 1.8036 - categorical_accuracy: 0.2485 - val_loss: 1.6195 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2020-09-1212_14_36.139518/model-00001-1.80357-0.24853-1.61954-0.22000.h5
Epoch 2/20
34/34 [==============================] - 58s 2s/step - loss: 1.8001 - categorical_accuracy: 0.2456 - val_loss: 1.5951 - val_categorical_accuracy: 0.2900

Epoch 00002: saving model to model_init_2020-09-1212_14_36.139518/model-00002-1.80014-0.24559-1.59511-0.29000.h5
Epoch 3/20
34/34 [==============================] - 59s 2s/step - loss: 1.7293 - categorical_accuracy: 0.2618 - val_loss: 1.5621 - val_categorical_accuracy: 0.2700

Epoch 00003: saving model to model_init_2020-09-1212_14_36.139518/model-00003-1.72931-0.26176-1.56208-0.27000.h5
Epoch 4/20
34/34 [==============================] - 59s 2s/step - loss: 1.6812 - categorical_accuracy: 0.2765 - val_loss: 1.5815 - val_categorical_accuracy: 0.3100

Epoch 00004:

#### <font color=blue>With batch size 20 and epoch = 10 and 16 images the accuracy was for train : 91% and val : 80% </font>

#### <font color=blue>With batch size 20 and epoch = 10 and 24 images the accuracy was for train : 53% and val : 28% </font>

#### <font color=blue>With batch size 20 and epoch = 20 and 16 images the accuracy was for train : 47% and val : 53% and time : 48 mins </font>

#### <font color=blue>With batch size 20 and epoch = 20 and 24 images(100,100) the accuracy was for train : 42% and val : 49%  </font>

### <span style ='background:yellow'> Model 9: Adding additional layer in the GRU </span>

In [79]:
model = Sequential()
model.add(TimeDistributed(Conv2D(8,kernel_size=(3,3), padding='same', activation='relu'),input_shape=(16,120,120,3)))   #### 30 in case of full , image size 120,120
        
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(GRU(32))
model.add(Dropout(0.5))
        
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_54 (TimeDis (None, 16, 120, 120, 8)   224       
_________________________________________________________________
time_distributed_55 (TimeDis (None, 16, 120, 120, 8)   32        
_________________________________________________________________
time_distributed_56 (TimeDis (None, 16, 60, 60, 8)     0         
_________________________________________________________________
time_distributed_57 (TimeDis (None, 16, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_58 (TimeDis (None, 16, 60, 60, 16)    64        
_________________________________________________________________
time_distributed_59 (TimeDis (None, 16, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_60 (TimeDis (None, 16, 14400)       

In [80]:
#batch_size = 20
num_epochs = 20  
img_cnt_idx=16
train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1




In [81]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10


C:\Users\arunb\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
C:\Users\arunb\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
C:\Users\arunb\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
C:\Users\arunb\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transfo

34/34 [==============================] - 90s 3s/step - loss: 1.8566 - categorical_accuracy: 0.2456 - val_loss: 1.4437 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2020-09-0913_35_07.815118/model-00001-1.85664-0.24559-1.44369-0.32000.h5
Epoch 2/10
34/34 [==============================] - 83s 2s/step - loss: 1.6792 - categorical_accuracy: 0.2691 - val_loss: 1.6309 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-09-0913_35_07.815118/model-00002-1.67921-0.26912-1.63089-0.21000.h5
Epoch 3/10
34/34 [==============================] - 83s 2s/step - loss: 1.6097 - categorical_accuracy: 0.2912 - val_loss: 1.5224 - val_categorical_accuracy: 0.2600

Epoch 00003: saving model to model_init_2020-09-0913_35_07.815118/model-00003-1.60968-0.29118-1.52241-0.26000.h5
Epoch 4/10
34/34 [==============================] - 88s 3s/step - loss: 1.5903 - categorical_accuracy: 0.2868 - val_loss: 1.7373 - val_categorical_accuracy: 0.2000

Epoch 00004:

#### <font color =blue>With batch size 20 and epoch = 10 and 16 images the accuracy was for train : 45% and val : 38% </font>

### <span style ='background:yellow'>Model 10 : Using Transfer learning</span>

In [9]:
from keras.applications import vgg16 as vgg

In [64]:
vgg_transfer = vgg.VGG16(weights='imagenet', include_top=False)

model = Sequential()

model.add(TimeDistributed(vgg_transfer,input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120     

for layer in model.layers:
     layer.trainable = False

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))


model.add(GRU(32))
model.add(Dropout(0.5))
        
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

58892288/58889256 [==============================] - 5s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_20 (TimeDis (None, 24, 3, 3, 512)     14714688  
_________________________________________________________________
time_distributed_21 (TimeDis (None, 24, 3, 3, 512)     2048      
_________________________________________________________________
time_distributed_22 (TimeDis (None, 24, 1, 1, 512)     0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 24, 512)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                52320     
_________________________________________________________________
dropout_22 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_26 (D

In [65]:
#batch_size = 20
num_epochs = 20  
img_cnt_idx=24

train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1




In [66]:
start_time = datetime.datetime.now()
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
end_time = datetime.datetime.now()
print("time taken" + str(end_time - start_time))

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = Epoch 1/20
20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 4s - loss: 1.7982 - categorical_accuracy: 0.2359

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 74s 2s/step - loss: 1.7997 - categorical_accuracy: 0.2397 - val_loss: 1.5863 - val_categorical_accuracy: 0.2700

Epoch 00001: saving model to model_init_2020-09-1212_14_36.139518/model-00001-1.79967-0.23971-1.58632-0.27000.h5
Epoch 2/20
34/34 [==============================] - 60s 2s/step - loss: 1.6195 - categorical_accuracy: 0.3235 - val_loss: 1.5261 - val_categorical_accuracy: 0.2600

Epoch 00002: saving model to model_init_2020-09-1212_14_36.139518/model-00002-1.61952-0.32353-1.52611-0.26000.h5
Epoch 3/20
34/34 [==============================] - 62s 2s/step - loss: 1.5671 - categorical_accuracy: 0.3191 - val_loss: 1.4526 - val_categorical_accuracy: 0.3800

Epoch 00003: saving model to model_init_2020-09-1212_14_36.139518/model-00003-1.56706-0.31912-1.45262-0.38000.h5
Epoch 4/20
34/34 [==============================] - 61s 2s/step - loss: 1.4662 - categorical_accuracy: 0.3662 - val_loss: 1.4117 - val_categorical_accuracy: 0.4100

Epoch 00004:

####  <font color=blue>With batch size 20 and epoch = 10 and 16 images the accuracy was for train : 72% and val : 71% </font>

#### <font color=blue>With batch size 20 and epoch = 20 and 24 images(100,100) the accuracy was for train : 88% and val : 58% </font>

### <span style ='background:yellow'>Model 11 : Using Transfer learning (removed dense layer at end)</span>

In [13]:
vgg_transfer = vgg.VGG16(weights='imagenet', include_top=False)

model = Sequential()

model.add(TimeDistributed(vgg_transfer,input_shape=(16,100,100,3)))   #### 30 in case of full , image size 120,120     

for layer in model.layers:
     layer.trainable = False

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))


model.add(GRU(32))
       
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_5 (TimeDist (None, 16, 3, 3, 512)     14714688  
_________________________________________________________________
time_distributed_6 (TimeDist (None, 16, 3, 3, 512)     2048      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 16, 1, 1, 512)     0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 16, 512)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                52320     
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 165       
Total para

In [14]:
#batch_size = 20
num_epochs = 20  
img_cnt_idx=16

train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1


In [15]:
start_time = datetime.datetime.now()
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
end_time = datetime.datetime.now()
print("time taken" + str(end_time - start_time))

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train Epoch 1/20; batch size = 20



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 2s - loss: 1.8556 - categorical_accuracy: 0.2500

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 49s 1s/step - loss: 1.8447 - categorical_accuracy: 0.2485 - val_loss: 1.4537 - val_categorical_accuracy: 0.3600

Epoch 00001: saving model to model_init_2020-09-1216_31_30.733418/model-00001-1.84470-0.24853-1.45375-0.36000.h5
Epoch 2/20
34/34 [==============================] - 41s 1s/step - loss: 1.3863 - categorical_accuracy: 0.4412 - val_loss: 1.2970 - val_categorical_accuracy: 0.4700

Epoch 00002: saving model to model_init_2020-09-1216_31_30.733418/model-00002-1.38629-0.44118-1.29704-0.47000.h5
Epoch 3/20
34/34 [==============================] - 42s 1s/step - loss: 1.1058 - categorical_accuracy: 0.5706 - val_loss: 1.2188 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_init_2020-09-1216_31_30.733418/model-00003-1.10578-0.57059-1.21875-0.55000.h5
Epoch 4/20
34/34 [==============================] - 42s 1s/step - loss: 0.9808 - categorical_accuracy: 0.6265 - val_loss: 1.1843 - val_categorical_accuracy: 0.5700

Epoch 00004:

####  <font color=blue>With batch size 20 and epoch = 20 and 24 images(100,100) the accuracy was for train : 96% and val : 52% </font>

####  <font color=blue>With batch size 20 and epoch = 20 and 16 images(100,100) the accuracy was for train : 97.8% and val : 61% </font>

### <span style ='background:yellow'>Model 12 :Use transfer Learning Second model </span>

In [67]:
from keras.applications import ResNet50 

In [68]:
resnet_transfer = ResNet50(weights='imagenet', include_top=False)

model = Sequential()

model.add(TimeDistributed(resnet_transfer,input_shape=(24,100,100,3)))   #### 30 in case of full , image size 120,120     

for layer in model.layers:
     layer.trainable = False

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))


model.add(GRU(32))
model.add(Dropout(0.3))
        
       
model.add(Dense(5, activation='softmax'))

optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 7s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_24 (TimeDis (None, 24, 3, 3, 2048)    23587712  
_________________________________________________________________
time_distributed_25 (TimeDis (None, 24, 3, 3, 2048)    8192      
_________________________________________________________________
time_distributed_26 (TimeDis (None, 24, 1, 1, 2048)    0         
_________________________________________________________________
time_distributed_27 (TimeDis (None, 24, 2048)          0         
_________________________________________________________________
gru_3 (GRU)                  (None, 32)                199776    
_________________________________________________________________
dropout_24 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_28 (D

In [69]:
#batch_size = 20
num_epochs = 20  
img_cnt_idx=24

train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1




In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 1.5672 - categorical_accuracy: 0.3547

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 73s 2s/step - loss: 1.5225 - categorical_accuracy: 0.3779 - val_loss: 1.9993 - val_categorical_accuracy: 0.1600

Epoch 00001: saving model to model_init_2020-09-1212_14_36.139518/model-00001-1.52253-0.37794-1.99935-0.16000.h5
Epoch 2/20
34/34 [==============================] - 56s 2s/step - loss: 0.8373 - categorical_accuracy: 0.7074 - val_loss: 2.1090 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2020-09-1212_14_36.139518/model-00002-0.83730-0.70735-2.10898-0.16000.h5
Epoch 3/20
34/34 [==============================] - 62s 2s/step - loss: 0.5695 - categorical_accuracy: 0.8250 - val_loss: 1.8449 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2020-09-1212_14_36.139518/model-00003-0.56954-0.82500-1.84493-0.25000.h5
Epoch 4/20
34/34 [==============================] - 62s 2s/step - loss: 0.4225 - categorical_accuracy: 0.8809 - val_loss: 2.6462 - val_categorical_accuracy: 0.1600

Epoch 00004:

####  <font color=blue>With batch size 20 and epoch = 10 and 16 images the accuracy was for train : 99% and val : 24% </font>

####  <font color=blue>With batch size 20 and epoch = 10 and 16 images the accuracy was for train : 99% and val : 24% </font>

## Final Model

In [10]:
from keras.models import load_model

In [11]:
model = load_model('./model_init_2020-09-1306_06_01.454275/model-00040-0.19021-0.93676-0.51330-0.85000.h5')
optimiser = optimizers.Adam()                                   #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_20 (Conv3D)           (None, 24, 100, 100, 8)   200       
_________________________________________________________________
batch_normalization_19 (Batc (None, 24, 100, 100, 8)   32        
_________________________________________________________________
activation_20 (Activation)   (None, 24, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 12, 50, 50, 8)     0         
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 12, 50, 50, 16)    1040      
_________________________________________________________________
batch_normalization_20 (Batc (None, 12, 50, 50, 16)    64        
_________________________________________________________________
activation_21 (Activation)   (None, 12, 50, 50, 16)    0         
__________

In [14]:
#batch_size = 20
num_epochs = 20  
img_cnt_idx=24

train_generator = generator(train_path, train_doc, batch_size,img_cnt_idx)
val_generator = generator(val_path, val_doc, batch_size,img_cnt_idx)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1




In [15]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Source path =  ./Project_data/train ; batch size = 20
./Project_data/val ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 10s - loss: 0.5706 - categorical_accuracy: 0.8203

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 175s 5s/step - loss: 0.5605 - categorical_accuracy: 0.8221 - val_loss: 0.6284 - val_categorical_accuracy: 0.7700

Epoch 00001: saving model to model_init_2020-09-1316_06_46.594852/model-00001-0.56052-0.82206-0.62843-0.77000.h5
Epoch 2/20
34/34 [==============================] - 61s 2s/step - loss: 0.3644 - categorical_accuracy: 0.8779 - val_loss: 0.7985 - val_categorical_accuracy: 0.7800

Epoch 00002: saving model to model_init_2020-09-1316_06_46.594852/model-00002-0.36437-0.87794-0.79852-0.78000.h5
Epoch 3/20
34/34 [==============================] - 60s 2s/step - loss: 0.2407 - categorical_accuracy: 0.9206 - val_loss: 0.7865 - val_categorical_accuracy: 0.7600

Epoch 00003: saving model to model_init_2020-09-1316_06_46.594852/model-00003-0.24070-0.92059-0.78654-0.76000.h5
Epoch 4/20
34/34 [==============================] - 61s 2s/step - loss: 0.2410 - categorical_accuracy: 0.9191 - val_loss: 0.8665 - val_categorical_accuracy: 0.8000

Epoch 00004